In [328]:
import numpy
import pandas as pd
import json

from geopy.geocoders import Nominatim

import matplotlib as cm
import matplotlib as colors

from sklearn.cluster import KMeans

import folium

print ("Library Loaded")

Library Loaded


In [329]:
trnto_df=pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_in_Los_Angeles_County,_California', flavor='html5lib')
trnto_df[0].to_csv('trnto1.csv')
trnto_data=pd.read_csv('trnto1.csv')

print (trnto_data.shape)
trnto_data.head()

(88, 4)


,Unnamed: 0,City,Date incorporated,Population as of(2010 Census)
0,0,Agoura Hills,"December 8, 1982",20330
1,1,Alhambra,"July 11, 1903",83653
2,2,Arcadia,"August 5, 1903",56364
3,3,Artesia,"May 29, 1959",16522
4,4,Avalon,"June 26, 1913",3728


In [330]:
#dropped date inc and pop and unmaned from list 
trnto_data.drop(['Unnamed: 0','Date incorporated','Population as of(2010 Census)'], axis=1, inplace=True)
print(trnto_data.shape)
trnto_data.head()



(88, 1)


,City
0,Agoura Hills
1,Alhambra
2,Arcadia
3,Artesia
4,Avalon


In [331]:
#get zip codes from http://www.laalmanac.com/communications/cm02_communities.php and download zip as csv file
trnto1_df=pd.read_csv(r'C:\Users\adnan\Desktop\lazipcodes.csv')
print (trnto1_df.shape)
trnto1_df.head()

(88, 2)


,City/Community,Zip Code
0,Agoura Hills,91301
1,Alhambra,91801
2,Arcadia,91006
3,Artesia,90701
4,Avalon (PO Boxes),90704


In [332]:
import pgeocode
pgeocode.Nominatim('US')
geolocator = pgeocode.Nominatim('US')
postal_codes = trnto1_df['Zip Code'].tolist()
latitudes = []
longitudes = []
print("pgeocode downloaded")


pgeocode downloaded


In [333]:

for i, postal_codes in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {zip codes from df}')
    g = geolocator.query_postal_code(postal_codes)
    
    if not g.empty:
        #print(f'Postal Code {postal_codes} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)
        
print (len(City_codes),len(latitudes),len(longitudes))

trnto_data['City Latitude']=latitudes
trnto_data['City Longitude']=longitudes
trnto_data.head(88)

88 88 88


,City,City Latitude,City Longitude
0,Agoura Hills,34.1227,-118.7573
1,Alhambra,34.0914,-118.1293
2,Arcadia,34.1324,-118.0264
3,Artesia,33.8654,-118.0731
4,Avalon,33.3320,-118.3437
...,...,...,...
83,Walnut,34.0183,-117.8546
84,West Covina,34.0673,-117.9366
85,West Hollywood,34.1074,-118.3652
86,Westlake Village,34.1472,-118.8383


In [334]:
CLIENT_ID = '4MXLLZOVKLRCHM1PXPCXJW5PMUYGDJN0ODFFJATRPPKOSUPR' # your Foursquare ID
CLIENT_SECRET = '0NMMWU2MKDJ0BZQVRZ4CICVAA3VLN5FY4XUKNLG1DUMDKFZF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [335]:
dt_venues = getNearbyVenues(names = trnto_data['City'], latitudes=trnto_data['City Latitude'], longitudes=trnto_data['City Longitude'])
print(dt_venues.shape) 
dt_venues.head(1294)

Agoura Hills
Alhambra
Arcadia
Artesia
Avalon
Azusa
Baldwin Park
Bell
Bell Gardens
Bellflower
Beverly Hills
Bradbury
Burbank
Calabasas
Carson
Cerritos
Claremont
Commerce
Compton
Covina
Cudahy
Culver City
Diamond Bar
Downey
Duarte
El Monte
El Segundo
Gardena
Glendale
Glendora
Hawaiian Gardens
Hawthorne
Hermosa Beach
Hidden Hills
Huntington Park
Industry
Inglewood
Irwindale
La Cañada Flintridge
La Habra Heights
La Mirada
La Puente
La Verne
Lakewood
Lancaster
Lawndale
Lomita
Long Beach
Los Angeles
Lynwood
Malibu
Manhattan Beach
Maywood
Monrovia
Montebello
Monterey Park
Norwalk
Palmdale
Palos Verdes Estates
Paramount
Pasadena
Pico Rivera
Pomona
Rancho Palos Verdes
Redondo Beach
Rolling Hills
Rolling Hills Estates
Rosemead
San Dimas
San Fernando
San Gabriel
San Marino
Santa Clarita
Santa Fe Springs
Santa Monica
Sierra Madre
Signal Hill
South El Monte
South Gate
South Pasadena
Temple City
Torrance
Vernon
Walnut
West Covina
West Hollywood
Westlake Village
Whittier
(1278, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agoura Hills,34.1227,-118.7573,G.I.66 Productions Inc.,34.124039,-118.755886,Video Store
1,Agoura Hills,34.1227,-118.7573,Bwana Trail,34.121424,-118.753927,Trail
2,Agoura Hills,34.1227,-118.7573,Dead Man Overlook,34.123557,-118.761030,Scenic Lookout
3,Agoura Hills,34.1227,-118.7573,Medicine Woman Trail,34.118563,-118.757534,Trail
4,Alhambra,34.0914,-118.1293,85C Bakery Cafe,34.093411,-118.130214,Bakery
...,...,...,...,...,...,...,...
1273,Westlake Village,34.1472,-118.8383,The Pool,34.148767,-118.836492,Pool
1274,Westlake Village,34.1472,-118.8383,Das Boot,34.147147,-118.835218,Boat or Ferry
1275,Westlake Village,34.1472,-118.8383,Gym,34.146498,-118.842175,Gym / Fitness Center
1276,Westlake Village,34.1472,-118.8383,Triunfo Greenbelt,34.147256,-118.842786,Trail


In [336]:
dt_venues.groupby('Venue').count()
print('There are {} unique categories.'.format(len(dt_venues['Venue Category'].unique())))

There are 232 unique categories.


In [337]:
address = 'Los Angeles, CA'
latitudes=dt_venues['Venue Latitude']
longitudes=dt_venues['Venue Longitude']

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [338]:
import folium

map_la = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map for venue in Downtown toronto
for lat, lng, label in zip(dt_venues['Venue Latitude'], dt_venues['Venue Longitude'], dt_venues['Venue Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
  
map_la

In [339]:
# one hot encoding
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['City'] = dt_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
manhattan_onehot = dt_onehot[fixed_columns]

dt_onehot.head()

,ATM,Accessories Store,African Restaurant,American Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,City
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agoura Hills
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agoura Hills
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agoura Hills
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agoura Hills
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Alhambra


In [340]:
dt_onehot.shape

(1278, 233)

In [341]:
dt_grouped = dt_onehot.groupby('City').mean().reset_index(False)
dt_grouped

,City,ATM,Accessories Store,African Restaurant,American Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agoura Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
1,Alhambra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.031250,0.0,0.0,0.0,0.03125,0.0,0.03125,0.0,0.0
2,Arcadia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
3,Artesia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
4,Azusa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Vernon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
80,Walnut,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
81,West Covina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.034483,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
82,West Hollywood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0


In [342]:
dt_grouped.shape

(84, 233)

In [343]:
topfive_venues = 5 #top 5 venues in la county 88 cities

for hood in dt_grouped['City']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(topfive_venues))
    print('\n')

----Agoura Hills----
            venue  freq
0           Trail  0.50
1  Scenic Lookout  0.25
2     Video Store  0.25
3             ATM  0.00
4   Movie Theater  0.00


----Alhambra----
                venue  freq
0      Ice Cream Shop  0.06
1  Seafood Restaurant  0.06
2    Sushi Restaurant  0.06
3              Bakery  0.06
4           Bookstore  0.03


----Arcadia----
                  venue  freq
0  Fast Food Restaurant  0.09
1   Japanese Restaurant  0.04
2       Bubble Tea Shop  0.04
3                  Bank  0.04
4        Gymnastics Gym  0.04


----Artesia----
                  venue  freq
0             Gift Shop   1.0
1                   ATM   0.0
2  Other Great Outdoors   0.0
3             Multiplex   0.0
4                Museum   0.0


----Azusa----
                      venue  freq
0  Mediterranean Restaurant  0.15
1               Auto Garage  0.08
2                Restaurant  0.08
3           Thai Restaurant  0.08
4            Sandwich Place  0.08


----Baldwin Park----
         

In [344]:
def return_most_common_venues(row, topfive_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:topfive_venues]

In [345]:
import numpy as np

newtopten_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top 10 venues
columns = ['City']
for ind in np.arange(newtopten_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = dt_grouped['City']

for ind in np.arange(dt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], newtopten_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()



(84, 11)


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,Trail,Scenic Lookout,Video Store,ATM,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon
1,Alhambra,Ice Cream Shop,Seafood Restaurant,Sushi Restaurant,Bakery,Bookstore,Gastropub,Burmese Restaurant,Noodle House,Shipping Store,Café
2,Arcadia,Fast Food Restaurant,Japanese Restaurant,Bubble Tea Shop,Bank,Gymnastics Gym,Baseball Field,Gym,Tea Room,Taiwanese Restaurant,Mexican Restaurant
3,Artesia,Gift Shop,ATM,Other Great Outdoors,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub
4,Azusa,Mediterranean Restaurant,Auto Garage,Restaurant,Thai Restaurant,Sandwich Place,Gun Shop,Gym / Fitness Center,Park,Donut Shop,Mexican Restaurant


In [346]:
from sklearn.cluster import KMeans #Run k-means to cluster the County into clusters.

kclusters = 10

dt_grouped_clustering = dt_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([3, 1, 1, 4, 1, 9, 8, 8, 1, 1, 1, 1, 1, 2, 1, 5, 5, 1, 8, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 9, 7, 1, 1, 1, 1, 3, 1, 3, 7, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [347]:
dt_grouped_clustering.head()

,ATM,Accessories Store,African Restaurant,American Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.00000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.03125,0.0,0.0,0.0,0.03125,0.0,0.03125,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00000,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0


In [348]:
import pandas 
# add clustering labels

dt_grouped_clustering['Cluster Labels']= kmeans.labels_

dt_merged = trnto_data.join([dt_grouped_clustering['Cluster Labels']])

# merge dt_grouped with dt to add latitude/longitude for each City
dt_merged = dt_merged.join(neighborhoods_venues_sorted.set_index('City'), on='City')

print(dt_merged.shape)
#dt_merged.drop(['Cluster Labels1'], axis=1, inplace=True)
dt_merged.head(4) # check the last columns!



(88, 14)


,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,34.1227,-118.7573,3.0,Trail,Scenic Lookout,Video Store,ATM,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon
1,Alhambra,34.0914,-118.1293,1.0,Ice Cream Shop,Seafood Restaurant,Sushi Restaurant,Bakery,Bookstore,Gastropub,Burmese Restaurant,Noodle House,Shipping Store,Café
2,Arcadia,34.1324,-118.0264,1.0,Fast Food Restaurant,Japanese Restaurant,Bubble Tea Shop,Bank,Gymnastics Gym,Baseball Field,Gym,Tea Room,Taiwanese Restaurant,Mexican Restaurant
3,Artesia,33.8654,-118.0731,4.0,Gift Shop,ATM,Other Great Outdoors,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub


In [350]:
import folium
from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors 

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [mcolors.to_hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['City Latitude'], dt_merged['City Longitude'], dt_merged['City'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1-7],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [351]:
#What has the most clusters
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 0]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Rosemead,34.0658,-118.0853,0.0,Convenience Store,Sporting Goods Shop,Pharmacy,Grocery Store,Fast Food Restaurant,Chinese Restaurant,Business Service,Furniture / Home Store,Pizza Place,Music Store


In [352]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 1]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Alhambra,34.0914,-118.1293,1.0,Ice Cream Shop,Seafood Restaurant,Sushi Restaurant,Bakery,Bookstore,Gastropub,Burmese Restaurant,Noodle House,Shipping Store,Café
2,Arcadia,34.1324,-118.0264,1.0,Fast Food Restaurant,Japanese Restaurant,Bubble Tea Shop,Bank,Gymnastics Gym,Baseball Field,Gym,Tea Room,Taiwanese Restaurant,Mexican Restaurant
4,Avalon,33.3320,-118.3437,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Bell Gardens,33.9653,-118.1515,1.0,ATM,Donut Shop,Mexican Restaurant,Fast Food Restaurant,Seafood Restaurant,Grocery Store,Fried Chicken Joint,Peruvian Restaurant,Persian Restaurant,Multiplex
9,Bellflower,33.8867,-118.1265,1.0,Sandwich Place,Pizza Place,BBQ Joint,Sushi Restaurant,Mexican Restaurant,Grocery Store,Dive Bar,Pharmacy,Fast Food Restaurant,Chinese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,South Pasadena,34.1109,-118.1547,1.0,Chinese Restaurant,Pizza Place,Grocery Store,Coffee Shop,Movie Theater,Spa,Frozen Yogurt Shop,Burger Joint,Shipping Store,Sandwich Place
80,Temple City,34.1016,-118.0537,1.0,Spa,Home Service,Health & Beauty Service,Pet Store,ATM,Organic Grocery,Multiplex,Museum,Music Store,Music Venue
81,Torrance,33.8307,-118.3832,1.0,French Restaurant,Motel,Mexican Restaurant,Cajun / Creole Restaurant,Clothing Store,Baseball Field,Bar,Café,Thai Restaurant,Sandwich Place
82,Vernon,33.9994,-118.2133,1.0,Rental Service,Sandwich Place,ATM,Other Great Outdoors,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant


In [353]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 2]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Calabasas,34.1419,-118.6641,2.0,Soccer Field,Health & Beauty Service,Tennis Court,Golf Course,ATM,Optical Shop,Movie Theater,Multiplex,Museum,Music Store


In [354]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 3]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agoura Hills,34.1227,-118.7573,3.0,Trail,Scenic Lookout,Video Store,ATM,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon
60,Pasadena,34.1468,-118.1391,3.0,Coffee Shop,Pizza Place,American Restaurant,Hotel,Mexican Restaurant,Steakhouse,Thai Restaurant,Cosmetics Shop,Salad Place,Shopping Mall
62,Pomona,34.0812,-117.7362,3.0,Baseball Field,Mexican Restaurant,Electronics Store,Donut Shop,Business Service,New American Restaurant,Nightclub,Nail Salon,Other Repair Shop,Noodle House


In [355]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 4]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Artesia,33.8654,-118.0731,4.0,Gift Shop,ATM,Other Great Outdoors,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub


In [356]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 5]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Cerritos,33.8669,-118.0686,5.0,Food Truck,Other Great Outdoors,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub
16,Claremont,34.1092,-117.7183,5.0,ATM,Grocery Store,Pizza Place,Burger Joint,Soccer Field,Mexican Restaurant,Bar,Market,Theater,Garden


In [357]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 6]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,Long Beach,33.7706,-118.182,6.0,Mexican Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Karaoke Bar,American Restaurant,Hotel,Convenience Store,Thai Restaurant,Juice Bar


In [358]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 7]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Industry,34.0011,-118.0371,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,Monterey Park,34.0534,-118.1271,7.0,Liquor Store,Chinese Restaurant,Performing Arts Venue,ATM,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub
63,Rancho Palos Verdes,33.7515,-118.3670,7.0,Trail,Other Great Outdoors,Sporting Goods Shop,Pizza Place,Optical Shop,Motorsports Shop,Movie Theater,Multiplex,Museum,Music Store


In [359]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 8]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Baldwin Park,34.0964,-117.9682,8.0,Pool Hall,Business Service,Plaza,Other Great Outdoors,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant
7,Bell,33.9653,-118.1515,8.0,ATM,Donut Shop,Mexican Restaurant,Fast Food Restaurant,Seafood Restaurant,Grocery Store,Fried Chicken Joint,Peruvian Restaurant,Persian Restaurant,Multiplex
18,Compton,33.8748,-118.2402,8.0,Clothing Store,Food Truck,Food Court,Other Repair Shop,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub


In [360]:
dt_merged.loc[lambda dt_merged: dt_merged['Cluster Labels'] == 9]

,City,City Latitude,City Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Azusa,34.1407,-117.9567,9.0,Mediterranean Restaurant,Auto Garage,Restaurant,Thai Restaurant,Sandwich Place,Gun Shop,Gym / Fitness Center,Park,Donut Shop,Mexican Restaurant
54,Montebello,34.0133,-118.1130,9.0,Seafood Restaurant,American Restaurant,Poke Place,Mobile Phone Shop,Supermarket,Pharmacy,Rental Car Location,Middle Eastern Restaurant,Mexican Restaurant,Burger Joint


In [361]:
print("Cluster 1 is the most common cluster and has many Restaurants.\n")




Cluster 1 is the most common cluster and has many Restaurants.

